In [10]:
import requests
import time
import hashlib
from neo4j import GraphDatabase
import json
import re
from tqdm import tqdm
import pandas as pd


In [3]:
API_PUBLIC_KEY = "929b4b43b3ab59b27417573fcd5868b9"
API_PRIVATE_KEY = "27bc1b826da0c2def3525f69a2668e7834744938"
API_LIMIT = 100
API_OFFSET = 0

In [4]:
current_timestamp = time.time()
hash = hashlib.md5((str(current_timestamp)+API_PRIVATE_KEY+API_PUBLIC_KEY).encode())

In [6]:
response_API = requests.get('https://gateway.marvel.com:443/v1/public/comics?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s'%(API_LIMIT,API_OFFSET,hash.hexdigest(),str(current_timestamp)))
with open("test_chiamata_comics.json","w") as out_file:
  out_file.write(response_API.text)


In [9]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))

In [66]:

def add_character_node(tx, name, character_id):
  name = name.replace('"'," ")
  nome_senza_parentesi = re.sub("[()]", "", name)
  query_string = 'MERGE (c:character{name:"%s"})' % (nome_senza_parentesi, character_id)
  result = tx.run(query_string)
  if name == nome_senza_parentesi:
    query_string_variante = 'MERGE (c:character_variant{name:"%s", character_id:%s})' % (
        name, character_id)


In [67]:
def add_comic_node_and_link(tx, comic_title, comic_id, character_id):
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s})'%(comic_title, comic_id)
  create_link_query = "MATCH (a:character),(b:comic) WHERE a.character_id = %s AND b.comic_id = %s MERGE (a)-[r:Presente]->(b)"%(character_id, comic_id)
  print(create_comic_node_query)
  # print(create_link_query)
  node_creation_result = tx.run(create_comic_node_query)
  link_creation_result = tx.run(create_link_query)

In [68]:
def add_comic_node(tx, comic_title, comic_id):
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s})'%(comic_title, comic_id)
  node_creation_result = tx.run(create_comic_node_query)


Get all marvel comics characters

In [ ]:
results_ended = False
iteration_number = 0
driver = GraphDatabase.driver(
    "bolt://localhost:7687", auth=("neo4j", "1234qwer"))
API_OFFSET = 0
while not results_ended:
    print("********Starting iteration n. %s \n" % (str(iteration_number)))
    response_API = requests.get('https://gateway.marvel.com:443/v1/public/characters?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s' %
                                (API_LIMIT, API_OFFSET, hash.hexdigest(), str(current_timestamp)))
    response_JSON = json.loads(response_API.text)
    try:
        if len(response_JSON['data']['results']) != 0:
            for character in response_JSON['data']['results']:
                with driver.session() as session:
                    try:
                        session.write_transaction(
                            add_character_node, character['name'], character["id"])
                    except Exception:
                        print(Exception)
            API_OFFSET += API_LIMIT
            print("********Ending iteration n. %s \n" %
                  (str(iteration_number)))
            iteration_number += 1
        else:
            results_ended = True
    except Exception:
        print(Exception)


In [5]:
import pandas as pd
results_ended = False
iteration_number = 0

API_OFFSET = 0
character_list=[]
while not results_ended:
    print("********Starting iteration n. %s \n" % (str(iteration_number)))
    response_API = requests.get('https://gateway.marvel.com:443/v1/public/characters?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s' %
                                (API_LIMIT, API_OFFSET, hash.hexdigest(), str(current_timestamp)))
    response_JSON = json.loads(response_API.text)
    try:
        if len(response_JSON['data']['results']) != 0:
            for character in response_JSON['data']['results']:
                character_list.append(character)
            API_OFFSET += API_LIMIT
            print("********Ending iteration n. %s \n" %
                  (str(iteration_number)))
            iteration_number += 1
        else:
            results_ended = True
    except Exception:
        print(Exception)

lista_personaggi = pd.DataFrame(character_list, columns=[
                                "id", "name", "description", "modified", "thumbnail", "resourceURI", "comics", "series", "stories", "events", "urls"])

lista_personaggi.to_csv("./api_character_list.csv", index=False)

********Starting iteration n. 0 

********Ending iteration n. 0 

********Starting iteration n. 1 

********Ending iteration n. 1 

********Starting iteration n. 2 

********Ending iteration n. 2 

********Starting iteration n. 3 

********Ending iteration n. 3 

********Starting iteration n. 4 

********Ending iteration n. 4 

********Starting iteration n. 5 

********Ending iteration n. 5 

********Starting iteration n. 6 

********Ending iteration n. 6 

********Starting iteration n. 7 

********Ending iteration n. 7 

********Starting iteration n. 8 

********Ending iteration n. 8 

********Starting iteration n. 9 

********Ending iteration n. 9 

********Starting iteration n. 10 

********Ending iteration n. 10 

********Starting iteration n. 11 

********Ending iteration n. 11 

********Starting iteration n. 12 

********Ending iteration n. 12 

********Starting iteration n. 13 

********Ending iteration n. 13 

********Starting iteration n. 14 

********Ending iteration n. 14 



Get all Marvel Comics and link characters

In [ ]:
results_ended = False
iteration_number = 0
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))
API_OFFSET = 0
API_LIMIT = 100
response_JSON = None

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))
while not results_ended:
  response_API = requests.get('https://gateway.marvel.com:443/v1/public/comics?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s'%(API_LIMIT,API_OFFSET,hash.hexdigest(),str(current_timestamp)))
  response_JSON = json.loads(response_API.text)
  
  try:
      if len(response_JSON['data']['results']) != 0:
        for comic in response_JSON['data']['results']:
          characters_list = comic["characters"]["items"]
          if len(characters_list) > 0:
            for character in characters_list:
              character_id = character["resourceURI"].split("/")
              character_id = character_id[len(character_id) - 1]
              with driver.session() as session:
                try:
                  session.write_transaction(add_comic_node_and_link, comic['title'], comic["id"], character_id)
                except Exception:
                  print(Exception)
          else:
            with driver.session() as session:
              try:
                session.write_transaction(add_comic_node, comic['title'], comic["id"])
              except Exception:
                print(Exception)
        API_OFFSET += API_LIMIT
        print("********Ending iteration n. %s ***************\n" %(str(iteration_number)))
        iteration_number +=1
      else:
        results_ended = True
  except Exception:
          print(Exception)
  

In [20]:
from IPython.display import clear_output
results_ended = False
iteration_number = 101
API_OFFSET = 100 * iteration_number
API_LIMIT = 100
response_JSON = None
comic_list = []
while not results_ended and iteration_number <= 200:
# for i in tqdm(range(100)):
  clear_output(wait=False)
  print("******** Iteration number %s **********"%(iteration_number))
  response_API = requests.get('https://gateway.marvel.com:443/v1/public/comics?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s'%(API_LIMIT,API_OFFSET,hash.hexdigest(),str(current_timestamp)))
  response_JSON = json.loads(response_API.text)
  try:
      if response_JSON['data']['results']:
        for comic in response_JSON['data']['results']:
          comic_list.append(comic)
        
        #print("********Ending iteration n. %s ***************\n" %(str(iteration_number)))
        
        iteration_number +=1

        API_OFFSET = 100 * iteration_number
      else:
        results_ended = True
  except Exception as e:
          print(response_JSON)

lista_personaggi = pd.DataFrame(comic_list, columns=[
                                "id", "digitalId", "title", "issueNumber", "variantDescription", "description", "modified", "isbn", "upc", "diamondCode", "ean", "issn", "format", "pageCount", "textObjects", "resourceURI", "urls", "series", "variants", "collections", "collectedIssues", "dates", "prices", "thumbnail", "images", "creators", "characters", "stories", "events"])

lista_personaggi.to_csv("./api_comic_list2.csv", index=False)

  

******** Iteration number 200 **********


Merge Csv files

In [21]:
import os
import glob
import pandas as pd

path = "./"
all_files = glob.glob(os.path.join(path, "api_comic_list*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "merged.csv")

In [63]:
driver = GraphDatabase.driver(
    "bolt://localhost:7687", auth=("neo4j", "1234qwer"))
while not results_ended:

    try:
        if len(response_JSON['data']['results']) != 0:
            for comic in response_JSON['data']['results']:
                characters_list = comic["characters"]["items"]
                if len(characters_list) > 0:
                    for character in characters_list:
                        character_id = character["resourceURI"].split("/")
                        character_id = character_id[len(character_id) - 1]
                        with driver.session() as session:
                            try:
                                session.write_transaction(
                                    add_comic_node_and_link, comic['title'], comic["id"], character_id)
                            except Exception:
                                print(Exception)
                else:
                    with driver.session() as session:
                        try:
                            session.write_transaction(
                                add_comic_node, comic['title'], comic["id"])
                        except Exception:
                            print(Exception)
            API_OFFSET += API_LIMIT
            print("********Ending iteration n. %s ***************\n" %
                  (str(iteration_number)))
            iteration_number += 1
        else:
            results_ended = True
    except Exception:
        print(Exception)


MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)